In [5]:
import rasterio
import numpy as np
import joblib

In [6]:
rf_model = joblib.load('rf_model.pkl')

In [13]:
with rasterio.open('features_raster.tif') as src:
    bands = [src.read(i + 1) for i in range(8)]
    profile = src.profile  

bands = np.array(bands)
height, width = bands.shape[1], bands.shape[2]

In [14]:
X_pred = bands.reshape(8, -1).T

In [18]:
valid_mask = ~np.any(X_pred == -32768, axis=1)
X_pred_clean = X_pred[valid_mask]


In [21]:
y_proba = np.full(X_pred.shape[0], np.nan)
y_proba[valid_mask] = rf_model.predict_proba(X_pred_clean)[:, 1] 

proba_raster = y_proba.reshape(height, width)


C:\Users\tipua\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [22]:
profile.update(dtype=rasterio.float32, count=1)

with rasterio.open('predicted_As_probability.tif', 'w', **profile) as dst:
    dst.write(proba_raster.astype(rasterio.float32), 1)